In [19]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF 
import xarray as xr
import matplotlib.pyplot as plt

from scipy.ndimage import label
import os

import os 
import time

from renewable_data_load import *

from threshold_tools import *

# Configure dask for parallel execution
#import dask
#dask.config.set(scheduler='threads', num_workers=10)  # Adjust num_workers to your CPU cores

In [20]:
resource = "pv"
module = "utility"
domain = "d02"
variable = "cf"
frequency = "day"
simulation = "ec-earth3"
reference_gwl = 0.8
future_gwl = 2.0

window_size = 60

simulations = ["mpi-esm1-2-hr", "miroc6", "taiesm1", 'ec-earth3',]
simulations = ['ec-earth3']

# Get bounds for reference GWL period
WRF_sim_name = sim_name_dict[simulation]
model = WRF_sim_name.split("_")[1]
ensemble_member = WRF_sim_name.split("_")[2]
ref_start_year, ref_end_year = get_gwl_crossing_period(model, ensemble_member, reference_gwl)
fut_start_year, fut_end_year = get_gwl_crossing_period(model, ensemble_member, future_gwl)


In [15]:
# load drought mask
drought_mask_ds = xr.open_dataset(f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_drought_mask_only.nc")

In [16]:
# load drought_ds
threshold_file = f"../data/thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
drought_threshold_ds = xr.open_dataset(threshold_file)
ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
ren_ds = ren_ds.convert_calendar("noleap")

# For testing: limit to 10 years
ren_ds = ren_ds.isel(time=slice(0,365*20))

# For full processing: use all data
# ren_ds = ren_ds  # No slicing needed

# reshape array
ds_doy = ren_ds.copy(deep=True)
ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
ds_doy['year'] = ds_doy.time.dt.year
ds_doy = ds_doy.assign_coords(
    {"dayofyear":ds_doy.time.dt.dayofyear,
    "year":ds_doy.time.dt.year}
)
# reshape time dimension
ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()

drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()

# reshape back into daily timeseries
drought_ds = drought_ds.stack(time=['year','dayofyear'])
drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)

drought_ds = drought_ds.load()

In [11]:
# crop both datasets to reference and future GWL periods
drought_ds_ref = drought_ds.sel(time=slice(f"{ref_start_year}-01-01", f"{ref_end_year}-12-31"))
drought_ds_fut = drought_ds.sel(time=slice(f"{fut_start_year}-01-01", f"{fut_end_year}-12-31"))

drought_mask_ds_ref = drought_mask_ds.sel(time=slice(f"{ref_start_year}-01-01", f"{ref_end_year}-12-31"))
drought_mask_ds_fut = drought_mask_ds.sel(time=slice(f"{fut_start_year}-01-01", f"{fut_end_year}-12-31"))

In [18]:
ren_ds

<xarray.DataArray 'cf' (y: 320, x: 250, time: 7300)> Size: 5GB
dask.array<getitem, shape=(320, 250, 7300), dtype=float64, chunksize=(87, 42, 6026), chunktype=numpy.ndarray>
Coordinates:
    Lambert_Conformal  int32 4B 1
    landmask           (y, x) float32 320kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 320kB 23.35 23.39 23.43 ... 56.42 56.46
    lon                (y, x) float32 320kB -116.5 -116.4 ... -111.6 -111.5
  * time               (time) object 58kB 1981-01-01 00:00:00 ... 2000-12-31 ...
  * x                  (x) float64 2kB -4.638e+06 -4.629e+06 ... -2.397e+06
  * y                  (y) float64 3kB -2.441e+05 -2.351e+05 ... 2.627e+06
Attributes:
    data_type:             Gridded
    extended_description:  Ratio of power generated to panel nameplate capacity.
    frequency:             day
    grid_mapping:          Lambert_Conformal
    long_name:             capacity factor
    units:                 1

In [45]:
drought_ds_selection

<xarray.DataArray (y: 320, x: 250, time: 1825)> Size: 1GB
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.65625418,  0.87029167,  0.79207917, ...,  0.83216669,
          0.14095416,  0.52700002],
        [ 0.64167082,  0.87741668,  0.78912085, ...,  0.84584164,
          0.1326125 ,  0.58816665],
        [ 0.60897499,  0.88350002,  0.75985418, ...,  0.81491666,
          0.1285375 ,  0.65670833]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [ 0.1524    , -0.02375   , -0.0236375 , ...,  0.74386669,
          0.53501251,  0.63958335],
        [ 0.19986666, -0.021125  , -0.0208625 , ...,  0.73902497,
          0.55545835,  0.72183335],
        [ 0.20204583, -0.02108333, -0.01954583, ...,  0.69348331,
          0.58065832,  0.82845833]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.12314584, -0.011875  , -0.01126667, ...,  0.58608334,
          0.48957498,  0.61458335],
        [ 0.11860833, -0.00566667, -0.00555417, ...,  0.50916667,
          0.50377084,  0.69062499],
        [ 0.12318751, -0.00258333, -0.0017375 , ...,  0.39707499,
          0.51874582,  0.77829168]]], shape=(320, 250, 1825))
Coordinates:
  * x                  (x) float64 2kB -4.638e+06 -4.629e+06 ... -2.397e+06
  * y                  (y) float64 3kB -2.441e+05 -2.351e+05 ... 2.627e+06
    Lambert_Conformal  int32 4B 1
    landmask           (y, x) float32 320kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 320kB 23.35 23.39 23.43 ... 56.42 56.46
    lon                (y, x) float32 320kB -116.5 -116.4 ... -111.6 -111.5
    quantile           float64 8B 0.1
    year               (time) int64 15kB 1996 1996 1996 1996 ... 2000 2000 2000
    dayofyear          (time) int64 15kB 1 2 3 4 5 6 ... 360 361 362 363 364 365
  * time               (time) object 15kB 1996-01-01 00:00:00 ... 2000-12-31 ...
Attributes:
    data_type:             Gridded
    extended_description:  Ratio of power generated to turbine nameplate capa...
    frequency:             day
    grid_mapping:          Lambert_Conformal
    long_name:             capacity factor
    units:                 1

In [46]:
drought_mask_selection = drought_mask_ds_ref['drought_mask']
drought_ds_selection = drought_ds_ref

In [47]:

def calculate_all_drought_metrics(mask_1d, drought_values_1d):
    """
    Calculate all drought metrics for a 1D time series in a single pass.
    Returns three arrays with values at first day of each event, NaN elsewhere.
    
    Parameters:
    -----------
    mask_1d : 1D array
        Binary drought mask (1 = drought, 0 = no drought)
    drought_values_1d : 1D array
        Drought intensity values (from drought_ds)
    
    Returns:
    --------
    duration_1d : 1D array
        Duration at first day of each drought event, NaN elsewhere
    intensity_1d : 1D array
        Maximum intensity (minimum value) at first day of each drought event, NaN elsewhere
    magnitude_1d : 1D array
        Magnitude (sum of values) at first day of each drought event, NaN elsewhere
    """
    # Initialize output arrays with NaN
    duration_1d = np.full_like(mask_1d, np.nan, dtype=np.float32)
    intensity_1d = np.full_like(mask_1d, np.nan, dtype=np.float32)
    magnitude_1d = np.full_like(mask_1d, np.nan, dtype=np.float32)
    
    # Use scipy.ndimage.label to identify connected drought events (once!)
    labeled_array, num_events = label(mask_1d)
    
    # For each drought event, calculate all metrics in one pass
    for event_id in range(1, num_events + 1):
        # Find indices for this event
        event_indices = np.where(labeled_array == event_id)[0]
        
        if len(event_indices) > 0:
            # Get first index
            first_idx = event_indices[0]
            
            # Get values during this event
            event_values = drought_values_1d[event_indices]
            
            # Calculate all three metrics
            duration = len(event_indices)
            intensity = np.min(event_values)
            magnitude = np.sum(event_values)
            
            # Store all at first day
            duration_1d[first_idx] = duration
            intensity_1d[first_idx] = intensity
            magnitude_1d[first_idx] = magnitude
    
    return duration_1d, intensity_1d, magnitude_1d



start_time_total = time.time()


    
# Apply to all spatial locations using xr.apply_ufunc
duration, intensity, magnitude = xr.apply_ufunc(
        calculate_all_drought_metrics,
        drought_mask_selection,
        drought_ds_selection,
        input_core_dims=[['time'], ['time']],
        output_core_dims=[['time'], ['time'], ['time']],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[np.float32, np.float32, np.float32]
    )
    
    # Force computation before next chunk

duration  = duration .compute()
intensity  = intensity .compute()
magnitude  = magnitude .compute()
    
    # Add names
# duration.name = "drought_duration"
# intensity.name = "drought_intensity"
# magnitude.name = "drought_magnitude"
    
    
    # Save chunk to disk immediately to free memory
drought_stats_ds = xr.Dataset({
        'duration': duration ,
        'intensity': intensity ,
        'magnitude': magnitude 
    })
    
#output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_drought_stats.nc"


    # Explicitly delete chunk data to free memory
#del duration, intensity, magnitude, drought_stats_ds
#del drought_mask_selection, drought_ds_selection

total_elapsed = time.time() - start_time_total
print(f"\n{'='*60}")
print(f"Completed in {total_elapsed:.2f} seconds ({total_elapsed/60:.2f} minutes)")

print(f"{'='*60}")


Completed in 26.64 seconds (0.44 minutes)


In [55]:
drought_stats_ds

<xarray.Dataset> Size: 2GB
Dimensions:            (x: 250, y: 320, time: 1825)
Coordinates:
  * x                  (x) float64 2kB -4.638e+06 -4.629e+06 ... -2.397e+06
  * y                  (y) float64 3kB -2.441e+05 -2.351e+05 ... 2.627e+06
    Lambert_Conformal  int32 4B 1
    landmask           (y, x) float32 320kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 320kB 23.35 23.39 23.43 ... 56.42 56.46
    lon                (y, x) float32 320kB -116.5 -116.4 ... -111.6 -111.5
    quantile           float64 8B 0.1
    year               (time) int64 15kB 1996 1996 1996 1996 ... 2000 2000 2000
    dayofyear          (time) int64 15kB 1 2 3 4 5 6 ... 360 361 362 363 364 365
  * time               (time) object 15kB 1996-01-01 00:00:00 ... 2000-12-31 ...
Data variables:
    duration           (y, x, time) float32 584MB nan nan nan ... nan nan nan
    intensity          (y, x, time) float32 584MB nan nan nan ... nan nan nan
    magnitude          (y, x, time) float32 584MB nan nan nan ... nan nan nan

In [54]:
drought_stats_ds.duration.time.size / 365

5.0

In [ ]:
drought_stats_ds.duration.sum('time') / 3

<xarray.DataArray 'duration' (y: 320, x: 250)> Size: 320kB
array([[0.       , 0.       , 0.       , ..., 6.0333333, 6.0333333,
        6.0666666],
       [0.       , 0.       , 0.       , ..., 6.133333 , 5.866667 ,
        5.8333335],
       [0.       , 0.       , 0.       , ..., 6.0666666, 5.9      ,
        5.9      ],
       ...,
       [0.       , 0.       , 0.       , ..., 6.233333 , 6.4666667,
        6.4333334],
       [0.       , 0.       , 0.       , ..., 6.1      , 5.9333334,
        6.366667 ],
       [0.       , 0.       , 0.       , ..., 6.3      , 6.1666665,
        6.1      ]], shape=(320, 250), dtype=float32)
Coordinates:
  * x                  (x) float64 2kB -4.638e+06 -4.629e+06 ... -2.397e+06
  * y                  (y) float64 3kB -2.441e+05 -2.351e+05 ... 2.627e+06
    Lambert_Conformal  int32 4B 1
    landmask           (y, x) float32 320kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 320kB 23.35 23.39 23.43 ... 56.42 56.46
    lon                (y, x) float32 320kB -116.5 -116.4 ... -111.6 -111.5
    quantile           float64 8B 0.1
Attributes:
    resource:       windpower
    module:         onshore
    domain:         d02
    variable:       cf
    simulation:     taiesm1
    reference_gwl:  0.8
    description:    Binary drought mask: 1 = drought (below threshold), 0 = n...